# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:
- Practice your SQL knowledge


## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [1]:
import pandas as pd
pysqldf = lambda q: sqldf(q, globals())
import sqlite3 
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, and address for all customers in California with a credit limit greater than 25000.00. 

In [2]:
# For the first query, the boilerplate for getting 
#the query into a dataframe has been provided for you
c.execute("""SELECT contactfirstName, contactlastName, phone, addressline1, city, creditlimit 
             from customers
             where (state == 'CA') and (creditlimit >=2501.00)
             ;""")

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

contactFirstName contactLastName       phone               addressLine1  \
0             Susan          Nelson  4155551450            5677 Strong St.   
1             Julie          Murphy  6505555787  5557 North Pendale Street   
2              Juri       Hashimoto  6505556809          9408 Furth Circle   
3             Julie           Young  6265557265         78934 Hillside Dr.   
4              Mary           Young  3105552373           4097 Douglas Av.   
5           Valarie        Thompson  7605558146           361 Furth Circle   
6             Julie           Brown  6505551386            7734 Strong St.   
7             Brian        Chandler  2155554369           6047 Douglas Av.   
8               Sue           Frick  4085553659             3086 Ingle Ln.   
9             Steve        Thompson  3105553722          3675 Furth Circle   
10              Sue          Taylor  4155554312          2793 Furth Circle   

             city creditLimit  
0      San Rafael   210500.00  
1   San Francisco    64600.00  
2      Burlingame    84600.00  
3        Pasadena    90700.00  
4        Glendale    11000.00  
5       San Diego   105000.00  
6   San Francisco   105000.00  
7     Los Angeles    57700.00  
8        San Jose    77600.00  
9         Burbank    55400.00  
10       Brisbane    60300.00

#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [3]:
c.execute("""select state, avg(creditlimit)
                      from customers
                      group by 1;""")

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

state  avg(creditlimit)
0                      61839.726027
1              BC      89950.000000
2              CA      83854.545455
3              CT      57350.000000
4        Co. Cork          0.000000
5   Isle of Wight      93900.000000
6              MA      70755.555556
7              NH     114200.000000
8              NJ      43000.000000
9             NSW     100550.000000
10             NV      71800.000000
11             NY      89966.666667
12          Osaka      81200.000000
13             PA      84766.666667
14       Pretoria          0.000000
15     Queensland      51600.000000
16         Québec      48700.000000
17          Tokyo      94400.000000
18       Victoria      88800.000000

#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [6]:
c.execute("""select customername, customernumber, ordernumber, status, quantityordered
                      from customers
                      join orders using(customernumber) 
                      join orderdetails using(ordernumber)
                      ;""")

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

customerName customerNumber orderNumber   status quantityOrdered
0  Atelier graphique            103       10123  Shipped              26
1  Atelier graphique            103       10123  Shipped              34
2  Atelier graphique            103       10123  Shipped              46
3  Atelier graphique            103       10123  Shipped              50
4  Atelier graphique            103       10298  Shipped              32

#### Expected Output

<img src='images/expected-output-3.png'>

## HAVING and ORDER BY

Now, repeat the last query, but only get orders from customers that have a quantityOrdered value greater than 30. Sort the rows in ascending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [18]:
c.execute("""select customername, customernumber, ordernumber, status, quantityordered
                      from customers
                      join orders using(customernumber) 
                      join orderdetails using(ordernumber)
                      group by 5
                      having sum(quantityordered) >30
                      order by quantityordered;""")


df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

customerName customerNumber orderNumber    status  \
0       Tokyo Collectables, Ltd            398       10408   Shipped   
1  Australian Collectables, Ltd            471       10415  Disputed   
2        Euro+ Shopping Channel            141       10412   Shipped   
3      Signal Collectibles Ltd.            487       10149   Shipped   
4             Kelly's Gift Shop            496       10138   Shipped   

  quantityOrdered  
0              15  
1              18  
2              19  
3              20  
4              21

In [16]:
c.execute("""SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, od.quantityOrdered FROM Customers c JOIN Orders o 
ON c.customerNumber = o.customerNumber JOIN OrderDetails od ON od.orderNumber = o.orderNumber
GROUP BY od.quantityOrdered
HAVING SUM(od.quantityOrdered) > 10 
ORDER BY od.quantityOrdered ASC""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

customerName customerNumber orderNumber   status  \
0  Extreme Desk Decorations, Ltd            412       10418  Shipped   
1        Tekni Collectables Inc.            328       10401  On Hold   
2          Salzburg Collectables            382       10419  Shipped   
3      The Sharp Gifts Warehouse            450       10407  On Hold   
4        Tokyo Collectables, Ltd            398       10408  Shipped   

  quantityOrdered  
0              10  
1              11  
2              12  
3              13  
4              15

#### Expected Output

<img src='images/expected-output-4.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

In [19]:
c.execute("""select firstName, lastName, employeenumber, officecode
                    from employees
                    where officeCode in (select officeCode 
                                                from offices join employees using (officeCode)
                                                group by 1
                                                having count(employeeNumber) < 5)
                                         ;""")

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

firstName   lastName employeeNumber officeCode
0   William  Patterson           1088          6
1     Julie   Firrelli           1188          2
2     Steve  Patterson           1216          2
3  Foon Yue      Tseng           1286          3
4    George     Vanauf           1323          3

In [ ]:
c.execute("""select lastName, firstName, employeeNumber, officeCode
                    FROM employees
                    WHERE officeCode IN (SELECT officeCode 
                                                FROM offices 
                                                JOIN employees
                                                USING(officeCode)
                                                GROUP BY 1
                                                HAVING COUNT(employeeNumber) < 5
                                         );
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!